<img src="http://www3.ifmg.edu.br/site_campi/p/images/logos/Betim-Simples.png" width="700"></img>
<b>Engenharia de Controle e Automação - Redes neurais artificiais</b> - Prof. Leandro Freitas

### Lucas Pevidor Reis - 0024855 - TP3

# 1  Objetivo

Fazer a implementação de uma rede neural multilayer perceptron (MLP) para resolver o problema XOR comentado nas primeiras aulas. Deve ser implementado um algoritmo de retropropagação (backpropagation) manualmente para fazer o ajustes dos pesos e dos bias para uma rede neural estruturada como a seguir:

![MLP](https://i.imgur.com/K998YGR.png)


# 2 Aproximação da função lógica XOR utilizando uma rede neural MLP

## 2.1 Função  a ser estimada:

A função lógica XOR recebe 2 entradas binárias e retorna 0 caso elas sejam iguais, ou 1 caso sejam diferentes, conforme mostrado na tabela abaixo.

![XOR](https://i.imgur.com/wg4Ckhs.png)

Visto que a função não é linearmente separável, é necessário utilizar uma rede neural de múltiplas camadas para realizar sua aproximação. Neste caso, será utilizada uma rede neural de 2 camadas.

## 2.2 Implementação da rede neural

A rede neural foi implementada utilizando classes. A classe neurônio já havia sido implementada nos últimos trabalhos. Foram feitas algumas pequenas modificações, que serão mostradas a seguir. Também foram implementadas as classes Camada e Rede.

### Classe neurônio



In [119]:
import numpy as np
from matplotlib import pyplot as plt

class Neuronio:

    #Função de inicialização da classe
    def __init__(self, n_entradas, func_act, use_bias):
        #Se optar por utilizar bias, será adicionada uma coluna no final dos vetores de entrada com valor unitário.
        self.use_bias = use_bias        
        if use_bias:
            self.pesos = np.empty((n_entradas+1, 1), dtype=float)
        else:
            self.pesos = np.empty((n_entradas, 1), dtype=float)
        self.novosPesos = np.array([])
        self.pesosAnteriores = self.pesos

        self.func_act = func_act
        self.delta = 0        
    
    def reset(self):
        #Fazer pesos randomizados
        vetor = np.ones((len(self.pesos[:,0]), 1), dtype=float)
        for i in range(len(vetor[:,0])):
            vetor[i,0] = np.random.normal(scale=0.3)
        self.pesos = vetor
        self.pesosAnteriores = vetor
        self.novosPesos = np.array([])
    
    @staticmethod
    def fSigmoid(v):
        saida = 1/(1+np.e**(-v))
        return saida
    
    #Definição de entradas
    #Função utilizada para configurar o vetor de entrada interno da classe, juntamente com normalização, adição
    #da coluna de bias. Utilizar quando a média e desvpad para normalização ainda não foram definidos.
    def set_entradas_ant(self, entradas, normalizar):        
        if self.use_bias:
            vetor_1 = np.zeros([len(entradas[:,0]), 1]) + 1
            if normalizar:
                self.normalizar(entradas, False)
                self.vetor_entradas = np.append(self.entr_normalizada, vetor_1, axis=1)
            else:
                self.vetor_entradas = np.append(entradas, vetor_1, axis=1)
        else:
            if normalizar:                
                self.vetor_entradas = self.normalizar(entradas, False)
            else:
                self.vetor_entradas = entradas
    
    #Definir entradas após treinamento da rede
    #Função utilizada para configurar o vetor de entrada interno da classe, juntamente com normalização, adição
    #da coluna de bias. Utilizar quando a média e desvpad para normalização ainda JÁ foram definidos.
    def set_entradas_pós(self, entradas, normalizar):
        if self.use_bias:
            vetor_1 = np.zeros([len(entradas[:,0]), 1]) + 1
            if normalizar:            
                self.vetor_entradas = np.append(self.normalizar(entradas, True), vetor_1, axis=1)
            else:
                self.vetor_entradas = np.append(entradas, vetor_1, axis=1)
        else:
            if normalizar:
                self.vetor_entradas = self.normalizar(entradas, True)
            else:                    
                self.vetor_entradas = entradas
    
    def normalizar(self, entradas, utilizar_parametros_anteriores):
        if utilizar_parametros_anteriores == False:
            #Encontrar média e desvio padrão a partir das entradas fornecidas.
            #Executado antes do treinamento da rede
            self.media = np.empty(len(entradas[0,:]), dtype=float)
            self.desvpad = np.empty(len(entradas[0,:]), dtype=float)
            self.entr_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
        
            for i in range(0, len(entradas[0,:])):
                self.media[i] = np.mean(entradas[:,i])
                self.desvpad[i] = np.sqrt(np.var(entradas[:,i]))            
                self.entr_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]
            return self.entr_normalizada
        else:
            #Normaliza os dados de entrada fornecidos conforme a média e desvio padrão já encontrados anteriormente
            entr_nova_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
            for i in range(0, len(entradas[0,:])):                
                entr_nova_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]            
            
            return entr_nova_normalizada    
        
    def proc_saida(self, entradas, normalizado):
        if not normalizado:
            #Normalização das entradas utilizando média e desvpad já encontrados
            entr_nova_normalizada = self.normalizar(entradas, True)
        else:
            entr_nova_normalizada = entradas
        
        v = np.dot(entr_nova_normalizada, self.pesos)
        saida = np.empty([1,1])
        if(self.func_act == 'Bipolar'):    
            for i in range(len(v)):
                if(v[i,:] > 0):
                    saida = np.append(saida, [[1]], axis=0)
                else:
                    saida = np.append(saida, [[-1]], axis=0)
            self.saida = saida[1:]
            return saida[1:]
        
        elif(self.func_act == 'Linear'):            
            self.saida = v
            return saida

        elif(self.func_act == 'Sigmoid'): 
            self.saida = self.fSigmoid(v)
            return self.saida
        
    def treinar_rede(self, fator_aprendizado, saida_desejada, max_iterações):
        redeTreinada = False
        self.historicoAcertos = []
        self.historicoPesos = np.empty([len(self.pesos), 1])
        
        for i in range(0, max_iterações):
            if not redeTreinada:
                flagAlteração = False
                acertos = 0
                for j in range(len(self.vetor_entradas[:,0])):
                    #Testa as entradas até encontrar um erro
                    #Caso encontrado, continua testando as outras entradas para obter a quantidade de acertos para
                    #os pesos atuais. Em seguida atualiza os pesos e reinicia os testes.
                    saida_proc = self.proc_saida([self.vetor_entradas[j,:]], True)[0,0]
                    res = saida_desejada[j,0] - saida_proc
                    if res == 0 and self.func_act=='Bipolar':
                        acertos += 1
                    else:
                        if self.func_act == 'Linear' or self.func_act == 'Bipolar':
                            deltaW = fator_aprendizado * res * self.vetor_entradas[j,:]
                        elif self.func_act == 'Sigmoid':
                            deltaW = fator_aprendizado * res * self.vetor_entradas[j,:] * saida_proc * (1 - saida_proc)
                        flagAlteração = True                                                
                        
                        #Se a rede for bipolar, termina de checar a quantidade de acertos
                        #antes de atualizar os pesos
                        if self.func_act=='Bipolar':
                            for k in range(j+1, len(self.vetor_entradas[:,0])):
                                res = saida_desejada[k,0] - self.proc_saida([self.vetor_entradas[k,:]], True)[0,0]
                                if res == 0:
                                    acertos += 1
                        
                            #Salva os acertos e pesos
                            self.historicoAcertos.append(acertos)
                        self.historicoPesos = np.append(self.historicoPesos, self.pesos, axis=1)
                        
                        #Atualiza os pesos                               
                        self.pesos += np.transpose([deltaW])
                        if self.func_act == 'Bipolar':
                            break
                
                if flagAlteração == False:
                    redeTreinada = True
                    self.historicoAcertos.append(acertos)
                    self.historicoPesos = np.append(self.historicoPesos, self.pesos, axis=1)                
        
        self.historicoErros = []
        for i in range(len(self.historicoAcertos)):            
            self.historicoErros.append(len(self.vetor_entradas) - self.historicoAcertos[i])
        
        if redeTreinada == False:
            if self.func_act=='Bipolar':
                #Se não foi possível obter 100% de acerto durante as iterações
                #encontra e define como pesos os que obtiveram mais acertos
                indiceMax = self.historicoAcertos.index(max(self.historicoAcertos))                        
                self.pesos = np.transpose([self.historicoPesos[:, indiceMax+1]])
                print('Nº erros: {}'.format(len(self.vetor_entradas[:,0])-max(self.historicoAcertos)))

    def trocarPesos(self):
        self.pesosAnteriores = self.pesos
        self.pesos = self.novosPesos
        self.novosPesos = np.array([])

Relacionando a classe atual com implementação do neurônio realizada nos últimos trabalhos, foram realizadas as seguintes alterações:

**Função init**
~~~~python
def __init__(self, n_entradas, func_act, use_bias):
    #Se optar por utilizar bias, será adicionada uma coluna no final dos vetores de entrada com valor unitário.
    self.use_bias = use_bias        
    if use_bias:
        self.pesos = np.empty((n_entradas+1, 1), dtype=float)
    else:
        self.pesos = np.empty((n_entradas, 1), dtype=float)
    self.novosPesos = np.array([])
    self.pesosAnteriores = self.pesos

    self.func_act = func_act
    self.delta = 0    
~~~~

Na função init foram adicionadas as variáveis *novosPesos*, *pesosAnteriores* e *delta*. Estas variáveis foram adicionadas para o cálculo do $\delta$, utilizado no algoritmo de *backpropagation*.

**Função reset**
~~~~python
def reset(self):
    #Fazer pesos randomizados
    vetor = np.ones((len(self.pesos[:,0]), 1), dtype=float)
    for i in range(len(vetor[:,0])):
        vetor[i,0] = np.random.normal(scale=0.3)
    self.pesos = vetor
    self.pesosAnteriores = vetor
    self.novosPesos = np.array([])
~~~~

A função reset foi alterada para ao invés de configurar os pesos para 0, configurá-los para valores randomizados obtidos a partir de uma distribuição normal de escala 0,3.

**Função fSigmoid**
~~~~python
@staticmethod
def fSigmoid(v):
    saida = 1/(1+np.e**(-v))
    return saida
~~~~

A função de sigmóide logística que estava implementada diretamente no algoritmo de treinamento do neurônio foi trocada para um método estático, para permitir o acesso da mesma por fora da classe. O acesso é realizado pela classe *RedeNeural*, durante a execução do algoritmo de *backpropagation*.

**Função trocarPesos**
~~~~python
def trocarPesos(self):
    self.pesosAnteriores = self.pesos
    self.pesos = self.novosPesos
    self.novosPesos = np.array([])
~~~~

A função *trocarPesos* realiza a circulação dos pesos no neurônio, trocando o conteúdo da variável *pesos* para a *pesosAnteriores* e de *novosPesos* para *pesos*.

### Classe Camada

In [120]:
class Camada:
    def __init__(self, n_entradas, n_neuronios, func_act):
        self.n_neuronios = n_neuronios
        self.n_entradas = n_entradas
        self.func_act = func_act
        self.saidas = np.zeros([n_neuronios, 1])
        self.entradas = np.zeros([n_entradas, 1])

        self.neuronios = []
        for i in range(n_neuronios):
            self.neuronios.append(Neuronio(n_entradas, func_act, use_bias=True))
            self.neuronios[i].reset()
    
    def setEntradas(self):
        for i in range(self.n_neuronios):
            self.neuronios[i].set_entradas_ant(np.transpose(self.entradas), False)

    def proc_saida(self):
        self.setEntradas()
        for i in range(self.n_neuronios):
            self.saidas[i,0] = self.neuronios[i].proc_saida(self.neuronios[i].vetor_entradas, normalizado=True)

As camadas consistem em um conjunto de neurônios. Ela é definida utilizando os parâmetros:
1. Número de entradas: Número de entradas de cada neurônio na camada
1. Número de neurônios: Quantidade de neurônios presentes na camada
1. Função de ativação: Função de ativação de cada neurônio

As camadas possuem um vetor de dimensão $(m,1)$ para entradas e um vetor $(n,1)$ para saídas. Esses vetores são utilizados para interligar as camadas da rede neural. Os neurônios da camada são organizados em uma lista. Estes são instanciados sempre utilizando o *bias*, e logo em seguida são resetados, para que seus pesos sejam randomizados.

A função *setEntradas* é responsável por ler os dados no vetor de entradas da camada e inserí-los nas entradas de cada neurônio, utilizando o método *Neuronios.set_entradas_ant*.

A função *proc_saida* processa a saída de cada neurônio presente na camada e organiza o resultado no vetor de saídas da camada.

### Classe RedeNeural

In [121]:
class RedeNeural:
    def __init__(self, n_entradas, n_camadasOcultas, nn_camadasOcultas, n_saidas, func_act):
        self.n_entradas = n_entradas
        self.n_camadasOcultas = n_camadasOcultas
        self.n_saidas = n_saidas
        self.func_act=func_act
        self.nn_camadasOcultas = nn_camadasOcultas
        
        self.listaEntradas = np.zeros([0,0]) #Lista com todas as entradas a serem processadas
        self.entradas = np.zeros([n_entradas, 1]) #Vetor de entradas para jogar nas camadas
        self.saidas = np.zeros([n_saidas, 1]) #Vetor de saída p/ receber das camadas
        self.listaSaidas = []

        self.camadas = []
        #Primeira camada, 1 neurônio para cada entrada
        c = 0
        self.camadas.append(Camada(n_entradas, n_entradas, func_act))
        self.camadas[c].entradas = self.entradas
        c += 1
        #Primeira camada oculta, n_entradas igual a nº de entradas
        if n_camadasOcultas != 0:
            self.camadas.append(Camada(n_entradas, nn_camadasOcultas, func_act))
            self.camadas[c].entradas = self.camadas[c-1].saidas
            c += 1
            #Demais camadas ocultas
            for i in range(n_camadasOcultas-1):
                self.camadas.append(Camada(nn_camadasOcultas, nn_camadasOcultas, func_act))
                self.camadas[c].entradas = self.camadas[c-1].saidas
                c += 1
            self.camadas.append(Camada(nn_camadasOcultas, n_saidas, func_act))
        else:
            self.camadas.append(Camada(n_entradas, n_saidas, func_act))
        self.camadas[c].entradas = self.camadas[c-1].saidas
        self.saidas = self.camadas[c].saidas

        self.n_camadas = c+1
    
    def set_entradas_pós(self, entradas, normalizar):
        if normalizar:           
            vetorEntradas = self.normalizar(entradas, True)     
            for i in range(len(vetorEntradas)):
                self.entradas[i] = vetorEntradas[i]
        else:
            for i in range(len(entradas)):
                self.entradas[i] = entradas[i]
    
    def normalizar(self, entradas, utilizar_parametros_anteriores):
        if utilizar_parametros_anteriores == False:
            #Encontrar média e desvio padrão a partir das entradas fornecidas.
            #Executado antes do treinamento da rede
            self.media = np.empty(len(entradas[0,:]), dtype=float)
            self.desvpad = np.empty(len(entradas[0,:]), dtype=float)
            self.entr_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
        
            for i in range(0, len(entradas[0,:])):
                self.media[i] = np.mean(entradas[:,i])
                self.desvpad[i] = np.sqrt(np.var(entradas[:,i]))            
                self.entr_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]
            return self.entr_normalizada
        else:
            #Normaliza os dados de entrada fornecidos conforme a média e desvio padrão já encontrados anteriormente
            entr_nova_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
            for i in range(0, len(entradas[0,:])):                
                entr_nova_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]            

            return entr_nova_normalizada

    def proc_saidas(self, normalizar):        
        self.listaSaidas = []
        for k in range(len(self.listaEntradas)):
            self.set_entradas_pós(np.transpose([self.listaEntradas[k]]), normalizar)
            for i in range(self.n_camadas):
                self.camadas[i].proc_saida()
            self.listaSaidas.append(self.saidas)

    def proc_saida(self, indice, normalizar):
        self.listaSaidas = []
        self.set_entradas_pós(np.transpose([self.listaEntradas[indice]]), normalizar)
        for i in range(self.n_camadas):
            self.camadas[i].proc_saida()
        self.listaSaidas.append(self.saidas)

    def treinar_rede(self, saidas_desejadas, tx_aprendizado_inicial, annealing, tx_momento, n_iteracoes):        
        self.registroTreinamento = []
        #primeiro passo: Processar a saída da rede com os dados atuais. A listaEntradas deve estar normalizada e configurada
        for m in range(n_iteracoes):
            print('-----------------------------------')
            tx_aprendizado = tx_aprendizado_inicial/(1+(m/annealing))
            print('eta = {} ; m={}'.format(tx_aprendizado, m))
            for l in range(len(self.listaEntradas)):
                self.proc_saida(l, False)
                print('[{}] - [{}]'.format(self.entradas[:,0], self.saidas))
                print('k')
                self.registroTreinamento.append([self.entradas[:,0], self.saidas])
                #cálculo do delta k e primeiro ajuste de pesos p/ última camada
                for n in self.camadas[self.n_camadas-1].neuronios:
                    n.delta = n.saida[0][0]*(1-n.saida[0][0])*(saidas_desejadas[l][0]-n.saida[0][0])
                    momento = tx_momento * (n.pesos - n.pesosAnteriores)
                    n.novosPesos = n.pesos + np.transpose([tx_aprendizado*n.delta*(np.append(self.camadas[self.n_camadas-2].saidas, np.array([[1]])))]) + momento
                #cálculo do delta j e ajustes de pesos para camadas ocultas e primeira camada
                for c in range(self.n_camadas-2, -1, -1):
                    for k, n in enumerate(self.camadas[c].neuronios):
                        #cálculo do somatório de wkdk da camada seguinte
                        soma = 0
                        for i, n2 in enumerate(self.camadas[c+1].neuronios):
                            soma += n2.delta * n2.pesos[k, 0]
                        #cálculo do deltaJ
                        n.delta = n.saida[0][0]*(1-n.saida[0][0])*soma
                        momento = tx_momento * (n.pesos - n.pesosAnteriores)
                        n.novosPesos = n.pesos + np.transpose([tx_aprendizado*n.delta*(np.append(self.camadas[c].entradas, np.array([[1]])))]) + momento

                #Troca os pesos dos neuronios
                for i in range(len(self.camadas)):
                    for n in self.camadas[i].neuronios:
                        n.trocarPesos()

Para instanciar a *RedeNeural* devem ser fornecidos os parâmetros:
1. Número de entradas da rede neural
1. Número de camadas ocultas
1. Número de neurônios de cada camada oculta
1. Número de saídas
1. Função de ativação

Assim como as camadas, a rede neural tem vetores para armazenamento das entradas, denominado *entradas*, e saídas, denominado *saidas*. Além destes vetores, há 2 outros vetores para armazenamento de entradas e saídas:
1. **listaEntradas**: Neste vetor são armazenados os conjuntos de entradas a serem processadas pela rede neural, com cada coluna no vetor representando uma entrada e cada linha representando um conjunto de entradas.
1. **listaSaidas**: Armazena os conjuntos de saídas processadas pela rede neural, com cada coluna no vetor representando o resultado de uma saída, e cada linha representando o conjunto de saídas processados pela rede.

Na rede neural as camadas são organizadas em uma lista denominada *camadas*. A primeira camada é instanciada de forma que a quantidade de neurônios seja igual à quantidade de entradas da rede neural. Após ser instanciada, seu vetor de entradas é referenciado ao vetor de entradas da rede neural.
Em seguida são instanciadas as camadas ocultas. A primeira camada oculta é instanciada com número de entradas iguais ao número de saídas da primeira camada da rede. Suas entradas são referenciadas às saídas da camada anterior. As demais camadas ocultas são instanciadas com número de entradas iguais ao número de saídas das próprias camadas ocultas. Cada camada tem suas entradas referenciadas às saídas das camadas anteriores. Por fim, é instanciada a camada de saída da rede. Suas entradas são referenciadas às saídas da camada anterior e sua saída é referenciada ao vetor de saídas da rede neural.

Uma estrutura simplificada da rede neural está representada na figura abaixo. Por motivos de organização não foram representadas todas as conexões dos neurônios. Cada neurônio está conectado à todas as saídas da camada anterior.
![Strut](https://i.imgur.com/qQLPrjp.png)

O método *set_entradas_pós*, mostrado abaixo, é responsável por transferir as entradas especificadas em seu argumento para o vetor de entradas da rede neural. O processo é feito iterativamente em um loop *for*, visto que caso o procesos fosse realizado por atribuição direta a referência entre o vetor de entradas da rede neural e o vetor de entradas da primeira camada seria perdida.

~~~~python
def set_entradas_pós(self, entradas, normalizar):
    if normalizar:           
        vetorEntradas = self.normalizar(entradas, True)     
        for i in range(len(vetorEntradas)):
            self.entradas[i] = vetorEntradas[i]
    else:
        for i in range(len(entradas)):
            self.entradas[i] = entradas[i]
~~~~

A inserção de dados na rede neural deve ser realizada através do vetor *listaEntradas*, e os dados já devem estar normalizados. Caso não estejam, a classe *RedeNeural* possui o método *normalizar*, exatamente igual ao método implementado nos neurônios.

**Exemplo**

O código abaixo implementa uma rede neural com 2 entradas, 2 camadas ocultas com 5 neurônios em cada camada e 2 saídas. A função de ativação é sigmoidal. Como entrada, serão utilizados os dados apresentados na tabela abaixo. Como os dados não estão normalizados será necessário normalizá-los.
![Tabela](https://i.imgur.com/2PxXV7U.png)

~~~~python
rede = RedeNeural(2, 2, 5, 2, 'Sigmoid')
Entradas = np.array([[0,0],[0.5,0.3],[0.1,0.6],[1,2]])
rede.listaEntradas = rede.normalizar(Entradas, False)
~~~~

O parâmetro False na função normalizar faz com que seja definida uma nova média e desvio padrão a partir das entradas fornecidas para o processo de normalização.

**Processamento das saídas**

Para o processamento de saídas foram implementados dois métodos. O primeiro, denominado *proc_saidas*, processa as saídas da rede neural para todas as entradas presentes no vetor *listaEntradas*. O outro método, denominado *proc_saida*, processa as saídas da rede neural para somente uma entrada presente no vetor *listaEntradas*, especificada pelo parâmetro indice. O processamento das saídas é realizado da seguinte forma:
1. Remove os elementos já processados no vetor *listaSaidas*;
1. Configura o vetor *entradas* com o elemento escolhido do vetor *listaEntradas*;
1. Percorre as camadas, da primeira à última, processando suas saídas pelo método *Camada.proc_saida*. Como suas entradas e saídas estão interligadas basta chamar o método seguindo as ordens das camadas, que suas entradas serão atualizadas automaticamente no processamento da saída da camada anterior;
1. Adiciona a saída processada ao vetor *listaSaidas*.

**Algoritmo de treinamento**

O algoritmo de treinamento recebe como parâmetros:
1. Vetor de saídas desejadas;
1. Taxa de aprendizado inicial $\eta$;
1. Taxa de *annealing*;
1. Taxa de momento;
1. Número de iterações a serem executadas.

O processo de treinamento é realizado da seguinte forma:

1. Seleciona-se um conjunto de entradas para a rede neural e encontra-se as saídas $y(k)$, onde $k$ representa a última camada da rede neural, para a entrada fornecida.
1. Inicialmente atua-se na última camada:
  1. Com as saídas $y(k)$ já calculadas para cada neurônio da camada de saída, calcula-se o $\delta_k(n) = y(k)(1-y(k))(d(k)-y(k))$ para cada neurônio desta camada. O valor de $\delta_k$ é armazenado na variável *Neuronio.delta*.
  1. Calcula-se o momento para ajuste dos pesos de cada neurônio, dado por $m = \alpha(w(n)-w(n-1))$, onde $\alpha$ corresponde à taxa de momento especificada no argumento da função de treinamento.
  1. Para cada neurônio, $n.novosPesos = w(n) + \eta\delta_k y(k-1) + m$
    1. Obs: os pesos ainda não são atualizados pois os pesos das camadas anteriores ainda devem ser ajustados
1. Seguindo agora da penúltima camada para trás, sendo a camada corrente $i$ e a camada seguinte $j$:
  1. Para cada neurônio $n$ da camada $i$, realiza-se o somatório de $w_{jn}\delta_j$ para cada neurônio da camada $j$. Este somatório é armazenado na variável *soma*.
  1. O $\delta_i$ para o neurônio $n$ é então calculado por $\delta_i = y(n)(1-y(n))soma$
  1. Calcula-se o momento para cada neurônio da camada $i$
  1. Define-se os novos pesos para os neurônios da camada $i$
1. Troca-se os pesos dos neurônios chamando a função *Neuronio.trocarPesos* em cada neurônio

Em seguida o processo é repetido para cada conjunto de entradas no vetor *listaEntradas*, e repetido novamente pelo número de iterações definidos no argumento da função de treinamento. Para cada iteração, o valor da taxa de aprendizado $\eta$ é calculado pela equação:

$\eta = \frac{\eta_0}{1+\frac{n}{\tau}}$

Onde $\eta_0$ corresponde à taxa de aprendizado inicial, $n$ corresponde à iteração corrente e $\tau$ corresponde à taxa de *annealing*.

# 3 Implementação da função XOR utilizando a rede MLP

Inicialmente instancia-se a rede neural. Será utilizada uma rede com 2 entradas, 1 camada de entrada, 1 camada de saída e função de ativação sigmoidal.

In [122]:
redeXOR = RedeNeural(2, 0, 0, 1, 'Sigmoid')

As entradas e saídas desejadas utilizadas para treinamento serão definidas conforme apresentado na tabela abaixo:

![XOR](https://i.imgur.com/wg4Ckhs.png)

In [123]:
entradas = np.array([[0,0],[0,1],[1,0],[1,1]])
saidas_desejadas = np.array([[0],[1],[1],[0]])

Em seguida configura-se as entradas da rede neural para o vetor *entradas*. Como as entradas não estão normalizadas deve-se executar o método *normalizar*.

In [124]:
redeXOR.listaEntradas = redeXOR.normalizar(entradas, False)
print(redeXOR.listaEntradas)

[[-1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]


Com as entradas definidas, é realizado então o treinamento da rede. Para o treinamento, foi escolhida uma taxa de aprendizado $\eta = 30$, *annealing* igual a 600, taxa de momento de 0,1 e 2000 iterações. 

In [125]:
#redeXOR.treinar_rede(saidas_desejadas, 30, 600, 0.1, 500)

In [126]:
redeXOR = RedeNeural(2, 0, 0, 1, 'Sigmoid')
redeXOR.listaEntradas = redeXOR.normalizar(entradas, False)
print(redeXOR.listaEntradas)
redeXOR.treinar_rede(saidas_desejadas, 20, 500, 0.1, 2000)

[[-1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]
-----------------------------------
eta = 20.0 ; m=0
[[-1. -1.]] - [[[0.59472657]]]
k
[[-1.  1.]] - [[[0.01866164]]]
k
[[ 1. -1.]] - [[[0.02117954]]]
k
[[1. 1.]] - [[[0.05660672]]]
k
-----------------------------------
eta = 19.960079840319363 ; m=1
[[-1. -1.]] - [[[0.03233073]]]
k
[[-1.  1.]] - [[[0.04030439]]]
k
[[ 1. -1.]] - [[[0.10102831]]]
k
[[1. 1.]] - [[[0.5852251]]]
k
-----------------------------------
eta = 19.9203187250996 ; m=2
[[-1. -1.]] - [[[0.05033255]]]
k
[[-1.  1.]] - [[[0.02966735]]]
k
[[ 1. -1.]] - [[[0.07049936]]]
k
[[1. 1.]] - [[[0.23014629]]]
k
-----------------------------------
eta = 19.880715705765407 ; m=3
[[-1. -1.]] - [[[0.12851475]]]
k
[[-1.  1.]] - [[[0.07355499]]]
k
[[ 1. -1.]] - [[[0.28571468]]]
k
[[1. 1.]] - [[[0.90477044]]]
k
-----------------------------------
eta = 19.841269841269842 ; m=4
[[-1. -1.]] - [[[0.72583202]]]
k
[[-1.  1.]] - [[[0.09470437]]]
k
[[ 1. -1.]] - [[[0.30978461]]]
k
[[1. 1.]] - [[[0.9

k
-----------------------------------
eta = 17.152658662092627 ; m=83
[[-1. -1.]] - [[[0.02926545]]]
k
[[-1.  1.]] - [[[0.96997414]]]
k
[[ 1. -1.]] - [[[0.96562346]]]
k
[[1. 1.]] - [[[0.02851713]]]
k
-----------------------------------
eta = 17.123287671232877 ; m=84
[[-1. -1.]] - [[[0.02903474]]]
k
[[-1.  1.]] - [[[0.97019184]]]
k
[[ 1. -1.]] - [[[0.96588901]]]
k
[[1. 1.]] - [[[0.02831318]]]
k
-----------------------------------
eta = 17.094017094017094 ; m=85
[[-1. -1.]] - [[[0.02880998]]]
k
[[-1.  1.]] - [[[0.97040439]]]
k
[[ 1. -1.]] - [[[0.96614809]]]
k
[[1. 1.]] - [[[0.02811374]]]
k
-----------------------------------
eta = 17.064846416382252 ; m=86
[[-1. -1.]] - [[[0.02859092]]]
k
[[-1.  1.]] - [[[0.970612]]]
k
[[ 1. -1.]] - [[[0.96640098]]]
k
[[1. 1.]] - [[[0.02791865]]]
k
-----------------------------------
eta = 17.035775127768314 ; m=87
[[-1. -1.]] - [[[0.02837731]]]
k
[[-1.  1.]] - [[[0.97081485]]]
k
[[ 1. -1.]] - [[[0.9666479]]]
k
[[1. 1.]] - [[[0.02772776]]]
k
-----------

k
-----------------------------------
eta = 15.797788309636651 ; m=133
[[-1. -1.]] - [[[0.0220792]]]
k
[[-1.  1.]] - [[[0.97695166]]]
k
[[ 1. -1.]] - [[[0.97404479]]]
k
[[1. 1.]] - [[[0.02186383]]]
k
-----------------------------------
eta = 15.772870662460567 ; m=134
[[-1. -1.]] - [[[0.02198914]]]
k
[[-1.  1.]] - [[[0.97704127]]]
k
[[ 1. -1.]] - [[[0.97415175]]]
k
[[1. 1.]] - [[[0.02177737]]]
k
-----------------------------------
eta = 15.748031496062993 ; m=135
[[-1. -1.]] - [[[0.02190031]]]
k
[[-1.  1.]] - [[[0.97712969]]]
k
[[ 1. -1.]] - [[[0.97425725]]]
k
[[1. 1.]] - [[[0.02169204]]]
k
-----------------------------------
eta = 15.723270440251572 ; m=136
[[-1. -1.]] - [[[0.0218127]]]
k
[[-1.  1.]] - [[[0.97721694]]]
k
[[ 1. -1.]] - [[[0.97436133]]]
k
[[1. 1.]] - [[[0.02160783]]]
k
-----------------------------------
eta = 15.698587127158556 ; m=137
[[-1. -1.]] - [[[0.02172628]]]
k
[[-1.  1.]] - [[[0.97730305]]]
k
[[ 1. -1.]] - [[[0.97446402]]]
k
[[1. 1.]] - [[[0.02152472]]]
k
-----

[[-1. -1.]] - [[[0.01849693]]]
k
[[-1.  1.]] - [[[0.98054704]]]
k
[[ 1. -1.]] - [[[0.978311]]]
k
[[1. 1.]] - [[[0.01838806]]]
k
-----------------------------------
eta = 14.534883720930234 ; m=188
[[-1. -1.]] - [[[0.01844819]]]
k
[[-1.  1.]] - [[[0.98059637]]]
k
[[ 1. -1.]] - [[[0.97836916]]]
k
[[1. 1.]] - [[[0.01834033]]]
k
-----------------------------------
eta = 14.513788098693757 ; m=189
[[-1. -1.]] - [[[0.0183999]]]
k
[[-1.  1.]] - [[[0.98064524]]]
k
[[ 1. -1.]] - [[[0.97842677]]]
k
[[1. 1.]] - [[[0.01829304]]]
k
-----------------------------------
eta = 14.492753623188406 ; m=190
[[-1. -1.]] - [[[0.01835206]]]
k
[[-1.  1.]] - [[[0.98069368]]]
k
[[ 1. -1.]] - [[[0.97848386]]]
k
[[1. 1.]] - [[[0.01824617]]]
k
-----------------------------------
eta = 14.47178002894356 ; m=191
[[-1. -1.]] - [[[0.01830466]]]
k
[[-1.  1.]] - [[[0.98074167]]]
k
[[ 1. -1.]] - [[[0.97854042]]]
k
[[1. 1.]] - [[[0.01819973]]]
k
-----------------------------------
eta = 14.450867052023122 ; m=192
[[-1. -1.

[[ 1. -1.]] - [[[0.98116534]]]
k
[[1. 1.]] - [[[0.01603606]]]
k
-----------------------------------
eta = 13.315579227696405 ; m=251
[[-1. -1.]] - [[[0.01607576]]]
k
[[-1.  1.]] - [[[0.98300892]]]
k
[[ 1. -1.]] - [[[0.98120053]]]
k
[[1. 1.]] - [[[0.01600695]]]
k
-----------------------------------
eta = 13.297872340425531 ; m=252
[[-1. -1.]] - [[[0.01604646]]]
k
[[-1.  1.]] - [[[0.98303885]]]
k
[[ 1. -1.]] - [[[0.98123549]]]
k
[[1. 1.]] - [[[0.01597803]]]
k
-----------------------------------
eta = 13.280212483399735 ; m=253
[[-1. -1.]] - [[[0.01601737]]]
k
[[-1.  1.]] - [[[0.98306858]]]
k
[[ 1. -1.]] - [[[0.98127021]]]
k
[[1. 1.]] - [[[0.0159493]]]
k
-----------------------------------
eta = 13.26259946949602 ; m=254
[[-1. -1.]] - [[[0.01598846]]]
k
[[-1.  1.]] - [[[0.98309811]]]
k
[[ 1. -1.]] - [[[0.98130469]]]
k
[[1. 1.]] - [[[0.01592076]]]
k
-----------------------------------
eta = 13.245033112582782 ; m=255
[[-1. -1.]] - [[[0.01595976]]]
k
[[-1.  1.]] - [[[0.98312745]]]
k
[[ 1. -

...

[[1. 1.]] - [[[0.00814985]]]
k
-----------------------------------
eta = 4.690431519699813 ; m=1632
[[-1. -1.]] - [[[0.00816626]]]
k
[[-1.  1.]] - [[[0.99120514]]]
k
[[ 1. -1.]] - [[[0.99058669]]]
k
[[1. 1.]] - [[[0.00814849]]]
k
-----------------------------------
eta = 4.688232536333802 ; m=1633
[[-1. -1.]] - [[[0.0081649]]]
k
[[-1.  1.]] - [[[0.99120658]]]
k
[[ 1. -1.]] - [[[0.9905883]]]
k
[[1. 1.]] - [[[0.00814713]]]
k
-----------------------------------
eta = 4.686035613870666 ; m=1634
[[-1. -1.]] - [[[0.00816353]]]
k
[[-1.  1.]] - [[[0.99120801]]]
k
[[ 1. -1.]] - [[[0.9905899]]]
k
[[1. 1.]] - [[[0.00814577]]]
k
-----------------------------------
eta = 4.683840749414521 ; m=1635
[[-1. -1.]] - [[[0.00816217]]]
k
[[-1.  1.]] - [[[0.99120945]]]
k
[[ 1. -1.]] - [[[0.99059151]]]
k
[[1. 1.]] - [[[0.00814441]]]
k
-----------------------------------
eta = 4.681647940074906 ; m=1636
[[-1. -1.]] - [[[0.00816081]]]
k
[[-1.  1.]] - [[[0.99121088]]]
k
[[ 1. -1.]] - [[[0.99059311]]]
k
[[1. 1.]

k
-----------------------------------
eta = 4.522840343735865 ; m=1711
[[-1. -1.]] - [[[0.00806232]]]
k
[[-1.  1.]] - [[[0.9913145]]]
k
[[ 1. -1.]] - [[[0.9907089]]]
k
[[1. 1.]] - [[[0.00804486]]]
k
-----------------------------------
eta = 4.520795660036167 ; m=1712
[[-1. -1.]] - [[[0.00806105]]]
k
[[-1.  1.]] - [[[0.99131583]]]
k
[[ 1. -1.]] - [[[0.99071039]]]
k
[[1. 1.]] - [[[0.0080436]]]
k
-----------------------------------
eta = 4.518752824220515 ; m=1713
[[-1. -1.]] - [[[0.00805979]]]
k
[[-1.  1.]] - [[[0.99131716]]]
k
[[ 1. -1.]] - [[[0.99071188]]]
k
[[1. 1.]] - [[[0.00804234]]]
k
-----------------------------------
eta = 4.5167118337850045 ; m=1714
[[-1. -1.]] - [[[0.00805852]]]
k
[[-1.  1.]] - [[[0.99131849]]]
k
[[ 1. -1.]] - [[[0.99071337]]]
k
[[1. 1.]] - [[[0.00804108]]]
k
-----------------------------------
eta = 4.514672686230249 ; m=1715
[[-1. -1.]] - [[[0.00805726]]]
k
[[-1.  1.]] - [[[0.99131982]]]
k
[[ 1. -1.]] - [[[0.99071485]]]
k
[[1. 1.]] - [[[0.00803982]]]
k
-----

k
[[1. 1.]] - [[[0.00795916]]]
k
-----------------------------------
eta = 4.382120946538125 ; m=1782
[[-1. -1.]] - [[[0.00797516]]]
k
[[-1.  1.]] - [[[0.99140623]]]
k
[[ 1. -1.]] - [[[0.99081135]]]
k
[[1. 1.]] - [[[0.00795797]]]
k
-----------------------------------
eta = 4.380201489268506 ; m=1783
[[-1. -1.]] - [[[0.00797398]]]
k
[[-1.  1.]] - [[[0.99140748]]]
k
[[ 1. -1.]] - [[[0.99081274]]]
k
[[1. 1.]] - [[[0.00795679]]]
k
-----------------------------------
eta = 4.378283712784588 ; m=1784
[[-1. -1.]] - [[[0.00797279]]]
k
[[-1.  1.]] - [[[0.99140873]]]
k
[[ 1. -1.]] - [[[0.99081414]]]
k
[[1. 1.]] - [[[0.0079556]]]
k
-----------------------------------
eta = 4.3763676148796495 ; m=1785
[[-1. -1.]] - [[[0.0079716]]]
k
[[-1.  1.]] - [[[0.99140998]]]
k
[[ 1. -1.]] - [[[0.99081553]]]
k
[[1. 1.]] - [[[0.00795442]]]
k
-----------------------------------
eta = 4.374453193350831 ; m=1786
[[-1. -1.]] - [[[0.00797042]]]
k
[[-1.  1.]] - [[[0.99141123]]]
k
[[ 1. -1.]] - [[[0.99081692]]]
k
[[1.

-----------------------------------
eta = 4.23908435777872 ; m=1859
[[-1. -1.]] - [[[0.00788662]]]
k
[[-1.  1.]] - [[[0.99149946]]]
k
[[ 1. -1.]] - [[[0.99091539]]]
k
[[1. 1.]] - [[[0.00786969]]]
k
-----------------------------------
eta = 4.23728813559322 ; m=1860
[[-1. -1.]] - [[[0.00788551]]]
k
[[-1.  1.]] - [[[0.99150063]]]
k
[[ 1. -1.]] - [[[0.9909167]]]
k
[[1. 1.]] - [[[0.00786858]]]
k
-----------------------------------
eta = 4.235493434985176 ; m=1861
[[-1. -1.]] - [[[0.0078844]]]
k
[[-1.  1.]] - [[[0.9915018]]]
k
[[ 1. -1.]] - [[[0.990918]]]
k
[[1. 1.]] - [[[0.00786747]]]
k
-----------------------------------
eta = 4.233700254022015 ; m=1862
[[-1. -1.]] - [[[0.00788329]]]
k
[[-1.  1.]] - [[[0.99150297]]]
k
[[ 1. -1.]] - [[[0.99091931]]]
k
[[1. 1.]] - [[[0.00786636]]]
k
-----------------------------------
eta = 4.231908590774439 ; m=1863
[[-1. -1.]] - [[[0.00788218]]]
k
[[-1.  1.]] - [[[0.99150414]]]
k
[[ 1. -1.]] - [[[0.99092061]]]
k
[[1. 1.]] - [[[0.00786526]]]
k
------------

[[ 1. -1.]] - [[[0.9910067]]]
k
[[1. 1.]] - [[[0.00779218]]]
k
-----------------------------------
eta = 4.111842105263158 ; m=1932
[[-1. -1.]] - [[[0.00780784]]]
k
[[-1.  1.]] - [[[0.99158244]]]
k
[[ 1. -1.]] - [[[0.99100794]]]
k
[[1. 1.]] - [[[0.00779114]]]
k
-----------------------------------
eta = 4.110152075626798 ; m=1933
[[-1. -1.]] - [[[0.00780679]]]
k
[[-1.  1.]] - [[[0.99158355]]]
k
[[ 1. -1.]] - [[[0.99100916]]]
k
[[1. 1.]] - [[[0.00779009]]]
k
-----------------------------------
eta = 4.108463434675431 ; m=1934
[[-1. -1.]] - [[[0.00780574]]]
k
[[-1.  1.]] - [[[0.99158465]]]
k
[[ 1. -1.]] - [[[0.99101039]]]
k
[[1. 1.]] - [[[0.00778905]]]
k
-----------------------------------
eta = 4.1067761806981515 ; m=1935
[[-1. -1.]] - [[[0.0078047]]]
k
[[-1.  1.]] - [[[0.99158575]]]
k
[[ 1. -1.]] - [[[0.99101162]]]
k
[[1. 1.]] - [[[0.00778801]]]
k
-----------------------------------
eta = 4.105090311986864 ; m=1936
[[-1. -1.]] - [[[0.00780366]]]
k
[[-1.  1.]] - [[[0.99158685]]]
k
[[ 1. 

Percebe-se que a rede neural consegue aproximar uma função XOR. Porém, o sucesso desta aproximação depende muito da escolha dos pesos iniciais. A escolha de  um certo conjunto de pesos leva à uma ótima aproximação, mas caso os pesos escolhidos inicialmente não sejam apropriados a aproximação não ocorre.

In [128]:
print(redeXOR.registroTreinamento)

[[array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772303]])], [array([1., 1.]), array([[0.00772